In [189]:
import pandas as pd
import plotly.express as px
import seaborn as sns

In [190]:
df =  pd.read_csv('winemag-data_first150k.csv')
df = df.drop(columns='Unnamed: 0')

In [191]:
df_2 =  pd.read_csv('winemag-data-130k-v2.csv')
df_2 = df_2.drop(columns='Unnamed: 0')
 

In [192]:
df = df.merge(df_2,how='outer')
df

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234663,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...
234664,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Pinot Noir,Citation,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon)
234665,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Gewürztraminer,Domaine Gresser,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...
234666,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Pinot Gris,Domaine Marcel Deiss,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace)


Comprobamos la información de nuestro dataset

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234668 entries, 0 to 234667
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                234602 non-null  object 
 1   description            234668 non-null  object 
 2   designation            165151 non-null  object 
 3   points                 234668 non-null  int64  
 4   price                  216150 non-null  float64
 5   province               234602 non-null  object 
 6   region_1               195693 non-null  object 
 7   region_2               93236 non-null   object 
 8   variety                234667 non-null  object 
 9   winery                 234668 non-null  object 
 10  taster_name            116131 non-null  object 
 11  taster_twitter_handle  110642 non-null  object 
 12  title                  151122 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 15.2+ MB


Tenemos muchos NaN en varias columnas, decido borrar la columna region_2, taster_name_, taster_twitter_handle.

La columna region_2 tiene demasiados NaN y ademas tenemos la columna region_1 que nos aporta datos de la región principal del vino, sobre las columnas taster_name y taster_twitter_handle, queremos enfocarnos en datos del vino, son columnas sobre datos de los expertos en vino y su Twitter.

In [194]:
df = df.drop(columns=['region_2','taster_name','taster_twitter_handle'])

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234668 entries, 0 to 234667
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      234602 non-null  object 
 1   description  234668 non-null  object 
 2   designation  165151 non-null  object 
 3   points       234668 non-null  int64  
 4   price        216150 non-null  float64
 5   province     234602 non-null  object 
 6   region_1     195693 non-null  object 
 7   variety      234667 non-null  object 
 8   winery       234668 non-null  object 
 9   title        151122 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 12.5+ MB


Vamos a trabajar los NaN de region_1, algunos países no distribuyen su territorio iguales que otros, sus divisiones de territorio son diferente, nos apoyaremos en la columna de provincia para rellenar region_1

Comprobamos cuantos países distintos hay

In [196]:
df['country'].unique()

array(['US', 'Spain', 'France', 'Italy', 'New Zealand', 'Bulgaria',
       'Argentina', 'Australia', 'Portugal', 'Israel', 'South Africa',
       'Greece', 'Chile', 'Morocco', 'Romania', 'Germany', 'Canada',
       'Moldova', 'Hungary', 'Austria', 'Croatia', 'Slovenia', nan,
       'India', 'Turkey', 'Macedonia', 'Lebanon', 'Serbia', 'Uruguay',
       'Switzerland', 'Albania', 'Bosnia and Herzegovina', 'Brazil',
       'Cyprus', 'Lithuania', 'Japan', 'China', 'South Korea', 'Ukraine',
       'England', 'Mexico', 'Georgia', 'Montenegro', 'Luxembourg',
       'Slovakia', 'Czech Republic', 'Egypt', 'Tunisia', 'US-France',
       'Peru', 'Armenia'], dtype=object)

Comprobamos si hay nulls en region_1 de US

In [197]:
df[(df['country']=='US')]['region_1'].isnull().values.any()

True

In [198]:
us = df[df['country']=='US']
us = us[us['region_1'].isnull()]
us['winery'].unique()


array(['Huston', '3 Horse Ranch Vineyards', 'Hat Ranch', 'Coiled', 'Vale',
       'Fujishin', 'Telaya', 'Sawtooth', 'Cinder', 'Ste. Chapelle',
       "Ryker's Cellars", 'Arizona Stronghold', "Colter's Creek",
       'Koenig Vineyards', 'Bitner', 'Margerum', 'Sokol Blosser', 'Truro',
       'Snake River', 'Williamson Vineyard', 'David Hill',
       'Clearwater Canyon', 'Indian Creek', 'Torii Mor', "Cooper's Hawk",
       'Mount Pleasant Winery', "Z'IVO", 'Adirondack Winery', 'Jana',
       'Alba', 'Gruet', 'Biltmore Estate', 'Original House Wine',
       'Prophecy', 'J. Scott Cellars', 'Laissez Faire', 'Flynn', 'Ayoub',
       'Bending Branch', 'Rascal', 'Cereghino Smith', 'Masút',
       'Château Élan', 'Luminary', 'Pacific Rim', 'Oliver',
       'Great American Wine Company', 'Spicewood', 'Honora',
       'Panther Creek', 'Becker', 'Fall Creek', 'Chateau St. Croix',
       'BBQ Wine Company', 'Corvidae', 'Ken Wright', 'Copper Belt',
       'ENSO'], dtype=object)

Caldwell = Huston,Hat Ranch,Vale,Coiled, Fujishin,
Eagle = 3 Horse Ranch Vineyards
Boise

Empezamos por Perú, sólo tiene una provincia que pertenece a la región geografica de Perú de costa

In [199]:
df[(df['country']=='Peru')]

,country,description,designation,points,price,province,region_1,variety,winery,title
160451,Peru,Aromas of freshly laid asphalt and char-grille...,Valle del Sol,85,14.0,Ica,NaN,Tannat,Intipalka,Intipalka 2013 Valle del Sol Tannat (Ica)
164914,Peru,"Roasted and earthy smelling, with a pinch of i...",Halcón de la Viña,85,15.0,Ica,NaN,Malbec,Tacama,Tacama 2008 Halcón de la Viña Malbec (Ica)
165971,Peru,Apple and citrus aromas struggle to hold it to...,Gran Blanco,82,10.0,Ica,NaN,White Blend,Tacama,Tacama 2010 Gran Blanco White (Ica)
165976,Peru,"Hot and leathery up front, with stewy notes an...",Gran Tinto,81,10.0,Ica,NaN,Red Blend,Tacama,Tacama 2009 Gran Tinto Red (Ica)
171849,Peru,Earthy plum and blackberry aromas are crisp an...,Valle del Sol Reserva,85,17.0,Ica,NaN,Malbec-Merlot,Intipalka,Intipalka 2012 Valle del Sol Reserva Malbec-Me...
181570,Peru,"At first, this blend of Cabernet Sauvignon, Ta...",Valle del Sol No 1 Gran Reserva,85,68.0,Ica,NaN,Red Blend,Intipalka,Intipalka 2010 Valle del Sol No 1 Gran Reserva...
197582,Peru,"Earthy, warm and ripe on the bouquet, with a b...",Sinfonía Tannat-Malbec-Petit Verdot,86,20.0,Ica,NaN,Red Blend,Tacama,Tacama 2008 Sinfonía Tannat-Malbec-Petit Verdo...
198011,Peru,Applesauce aromas are sweet and dusty. Then in...,Semi-Seco Blanco,83,10.0,Ica,NaN,White Blend,Tacama,Tacama 2010 Semi-Seco Blanco White (Ica)
206886,Peru,Stone-fruit and yellow apple aromas come with ...,Valle del Sol,82,14.0,Ica,NaN,Chardonnay,Intipalka,Intipalka 2014 Valle del Sol Chardonnay (Ica)
206898,Peru,Jammy strawberry and raspberry aromas are lean...,Valle del Sol,81,14.0,Ica,NaN,Syrah,Intipalka,Intipalka 2013 Valle del Sol Syrah (Ica)


Rellenamos los NaN

In [200]:
#Aplicamos una doble mascara y rellenamos con fillna
df[(df['country']=='Peru') & (df['province']=='Ica')].fillna('Costa')

,country,description,designation,points,price,province,region_1,variety,winery,title
160451,Peru,Aromas of freshly laid asphalt and char-grille...,Valle del Sol,85,14.0,Ica,Costa,Tannat,Intipalka,Intipalka 2013 Valle del Sol Tannat (Ica)
164914,Peru,"Roasted and earthy smelling, with a pinch of i...",Halcón de la Viña,85,15.0,Ica,Costa,Malbec,Tacama,Tacama 2008 Halcón de la Viña Malbec (Ica)
165971,Peru,Apple and citrus aromas struggle to hold it to...,Gran Blanco,82,10.0,Ica,Costa,White Blend,Tacama,Tacama 2010 Gran Blanco White (Ica)
165976,Peru,"Hot and leathery up front, with stewy notes an...",Gran Tinto,81,10.0,Ica,Costa,Red Blend,Tacama,Tacama 2009 Gran Tinto Red (Ica)
171849,Peru,Earthy plum and blackberry aromas are crisp an...,Valle del Sol Reserva,85,17.0,Ica,Costa,Malbec-Merlot,Intipalka,Intipalka 2012 Valle del Sol Reserva Malbec-Me...
181570,Peru,"At first, this blend of Cabernet Sauvignon, Ta...",Valle del Sol No 1 Gran Reserva,85,68.0,Ica,Costa,Red Blend,Intipalka,Intipalka 2010 Valle del Sol No 1 Gran Reserva...
197582,Peru,"Earthy, warm and ripe on the bouquet, with a b...",Sinfonía Tannat-Malbec-Petit Verdot,86,20.0,Ica,Costa,Red Blend,Tacama,Tacama 2008 Sinfonía Tannat-Malbec-Petit Verdo...
198011,Peru,Applesauce aromas are sweet and dusty. Then in...,Semi-Seco Blanco,83,10.0,Ica,Costa,White Blend,Tacama,Tacama 2010 Semi-Seco Blanco White (Ica)
206886,Peru,Stone-fruit and yellow apple aromas come with ...,Valle del Sol,82,14.0,Ica,Costa,Chardonnay,Intipalka,Intipalka 2014 Valle del Sol Chardonnay (Ica)
206898,Peru,Jammy strawberry and raspberry aromas are lean...,Valle del Sol,81,14.0,Ica,Costa,Syrah,Intipalka,Intipalka 2013 Valle del Sol Syrah (Ica)


Alemania, vamos a comprobar cuantas provincias distintas hay.

In [201]:
df[df['country']=='Germany']['province'].unique()

array(['Mosel', 'Rheinhessen', 'Württemberg', 'Ahr', 'Rheingau', 'Nahe',
       'Baden', 'Franken', 'Pfalz', 'Mosel-Saar-Ruwer', 'Germany',
       'Rhein-Mosel', 'Mittelrhein', 'Landwein Rhein'], dtype=object)

Rellenamos cada región a partir de su provincia.


In [202]:
df[(df['country']=='Germany') & (df['province']=='Mosel')].fillna('Mosel-Saar-Ruwer')

,country,description,designation,points,price,province,region_1,variety,winery,title
385,Germany,"Juicy, sun-kissed grapefruits and apple blosso...",Graacher Himmelreich Kabinett,91,23.0,Mosel,Mosel-Saar-Ruwer,Riesling,Friedrich-Wilhelm-Gymnasium,Friedrich-Wilhelm-Gymnasium 2014 Graacher Himm...
386,Germany,"Juicy, sun-kissed grapefruits and apple blosso...",Graacher Himmelreich Kabinett,91,23.0,Mosel,Mosel-Saar-Ruwer,Riesling,Friedrich-Wilhelm-Gymnasium,Friedrich-Wilhelm-Gymnasium 2014 Graacher Himm...
470,Germany,"Apple, pear and stone-fruit flavors are vibran...",Bernkasteler Badstube Kabinett,90,31.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. H. Thanisch (Erben Thanisch),Dr. H. Thanisch (Erben Thanisch) 2014 Bernkast...
471,Germany,"Apple, pear and stone-fruit flavors are vibran...",Bernkasteler Badstube Kabinett,90,31.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. H. Thanisch (Erben Thanisch),Dr. H. Thanisch (Erben Thanisch) 2014 Bernkast...
472,Germany,"Apple, pear and stone-fruit flavors are vibran...",Bernkasteler Badstube Kabinett,90,31.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. H. Thanisch (Erben Thanisch),Dr. H. Thanisch (Erben Thanisch) 2014 Bernkast...
...,...,...,...,...,...,...,...,...,...,...
234423,Germany,"Barely there whiffs of honey, cantaloupe and t...",Mosel-Saar-Ruwer,88,11.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. Heidemanns-Bergweiler,Dr. Heidemanns-Bergweiler 2015 Riesling (Mosel)
234634,Germany,"Sun-kissed nectarine, tangerine and grapefruit...",Bernkasteler Lay Kabinett,91,28.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. Loosen,Dr. Loosen 2015 Bernkasteler Lay Kabinett Ries...
234635,Germany,"Dusty pollen and mineral notes lend spicy, sav...",Ürziger Würzgarten Kabinett,91,28.0,Mosel,Mosel-Saar-Ruwer,Riesling,Dr. Loosen,Dr. Loosen 2015 Ürziger Würzgarten Kabinett Ri...
234643,Germany,"Plump, clingy peach and honey notes are cut wi...",Dom,90,17.0,Mosel,Mosel-Saar-Ruwer,Riesling,Bischöfliche Weingüter Trier,Bischöfliche Weingüter Trier 2013 Dom Riesling...


In [203]:
df[(df['country']=='Germany') & (df['province']=='Rheinhessen')].fillna('Costa')

,country,description,designation,points,price,province,region_1,variety,winery,title
419,Germany,"Often herbal and a bit green in nature, Silvan...",Saulheimer Probstey Trocken,91,39.0,Rheinhessen,Costa,Silvaner,Thörle,Thörle 2013 Saulheimer Probstey Trocken Silvan...
420,Germany,"Often herbal and a bit green in nature, Silvan...",Saulheimer Probstey Trocken,91,39.0,Rheinhessen,Costa,Silvaner,Thörle,Thörle 2013 Saulheimer Probstey Trocken Silvan...
1223,Germany,Pretty floral and apricot aromas brighten savo...,Niersteiner Hipping vom Rottligenden Spätlese...,91,17.0,Rheinhessen,Costa,Riesling,Weingut Georg Albrecht Schneider,Costa
1349,Germany,"Delightfully fun and refreshing, this perlwein...",Perlwein Trocken,90,19.0,Rheinhessen,Costa,Müller-Thurgau,Fritz Müller,Fritz Müller NV Perlwein Trocken Müller-Thurga...
1350,Germany,"Delightfully fun and refreshing, this perlwein...",Perlwein Trocken,90,19.0,Rheinhessen,Costa,Müller-Thurgau,Fritz Müller,Fritz Müller NV Perlwein Trocken Müller-Thurga...
...,...,...,...,...,...,...,...,...,...,...
232889,Germany,"The cheery, cartoonish cat gracing the label o...",Costa,89,12.0,Rheinhessen,Costa,Riesling,Lucky Neko,Lucky Neko 2014 Riesling (Rheinhessen)
233073,Germany,Hints of summer strawberry and peach show prom...,Weinkellerei Auslese,85,14.0,Rheinhessen,Costa,White Blend,Leonard Kreusch,Leonard Kreusch 2016 Weinkellerei Auslese Whit...
233300,Germany,Zippy lemongrass and white grapefruit flavors ...,Spätlese,88,14.0,Rheinhessen,Costa,Riesling,Dr. Volkmar,Dr. Volkmar 2015 Spätlese Riesling (Rheinhessen)
233589,Germany,Fresh white peach and green melon flavors shin...,Costa,86,16.0,Rheinhessen,Costa,Riesling,Weingut Liebfrauenstift,Weingut Liebfrauenstift 2014 Riesling (Rheinhe...


In [204]:

df[(df['country']=='Germany') & (df['province']=='Württemberg')].fillna('Baden-Wurttemberg')

,country,description,designation,points,price,province,region_1,variety,winery,title
457,Germany,Ruddy black plums and blackberries scent this ...,Dry,90,25.0,Württemberg,Baden-Wurttemberg,Lemberger,Schnaitmann,Baden-Wurttemberg
1239,Germany,"Whiffs of blackberry, bramble and granite sedu...",Trocken,91,24.0,Württemberg,Baden-Wurttemberg,Lemberger,Grafen Neipperg,Baden-Wurttemberg
2318,Germany,Hints of menthol and bramble meld with rich bl...,Trocken,90,26.0,Württemberg,Baden-Wurttemberg,Spätburgunder,Grafen Neipperg,Grafen Neipperg 2011 Trocken Spätburgunder (Wü...
2906,Germany,Juicy bursts of ripe blackberry and blueberry ...,Trocken,89,23.0,Württemberg,Baden-Wurttemberg,Lemberger,Grafen Neipperg,Grafen Neipperg 2013 Trocken Lemberger (Württe...
2907,Germany,Juicy bursts of ripe blackberry and blueberry ...,Trocken,89,23.0,Württemberg,Baden-Wurttemberg,Lemberger,Grafen Neipperg,Grafen Neipperg 2013 Trocken Lemberger (Württe...
3103,Germany,Char and roasted coffee notes add toasty nuanc...,Neipperger,90,37.0,Württemberg,Baden-Wurttemberg,Spätburgunder,Grafen Neipperg,Grafen Neipperg 2011 Neipperger Spätburgunder ...
4932,Germany,"Muskateller is a powerhouse Muscat grape, choc...",Halbtrocken,87,26.0,Württemberg,Baden-Wurttemberg,Muskateller,Grafen Neipperg,Grafen Neipperg 2013 Halbtrocken Muskateller (...
4933,Germany,"Muskateller is a powerhouse Muscat grape, choc...",Halbtrocken,87,26.0,Württemberg,Baden-Wurttemberg,Muskateller,Grafen Neipperg,Grafen Neipperg 2013 Halbtrocken Muskateller (...
5249,Germany,"While dry in style, sumptuous black cherry and...",Trocken,88,25.0,Württemberg,Baden-Wurttemberg,Lemberger,Grafen Neipperg,Grafen Neipperg 2012 Trocken Lemberger (Württe...
5250,Germany,"While dry in style, sumptuous black cherry and...",Trocken,88,25.0,Württemberg,Baden-Wurttemberg,Lemberger,Grafen Neipperg,Grafen Neipperg 2012 Trocken Lemberger (Württe...


In [205]:
df[(df['country']=='Germany') & (df['province']=='Ahr')].fillna('Anbaugebiet')

,country,description,designation,points,price,province,region_1,variety,winery,title
481,Germany,"Quite rare in the U.S. market, this ripe richl...",Trocken,90,40.0,Ahr,Anbaugebiet,Früburgunder,Winzergenossenschaft Mayschoss-Altenahr,Winzergenossenschaft Mayschoss-Altenahr 2013 T...
115704,Germany,"Hints of caramel and vanilla, along with soft,...",Recher Herrenberg,89,125.0,Ahr,Anbaugebiet,Pinot Noir,Jean Stodden,Jean Stodden 2008 Recher Herrenberg Pinot Noir...
116362,Germany,"Feminine and pretty, with hints of rose, viole...",J,88,99.0,Ahr,Anbaugebiet,Pinot Noir,Jean Stodden,Anbaugebiet
117730,Germany,"While lithe on the nose, with understated red-...",Anbaugebiet,87,78.0,Ahr,Anbaugebiet,Pinot Noir,Jean Stodden,Jean Stodden 2010 Pinot Noir (Ahr)
119037,Germany,Firm tannins and notes of fresh dill and toast...,1479N Ruber,86,28.0,Ahr,Anbaugebiet,Spätburgunder,Nelles,Nelles 2009 1479N Ruber Spätburgunder (Ahr)
207420,Germany,Swathes of violet and lavender scent this pret...,Anbaugebiet,92,33.0,Ahr,Anbaugebiet,Spätburgunder,Meyer-Näkel,Meyer-Näkel 2013 Spätburgunder (Ahr)
211890,Germany,Spice and black plums are touched with hints o...,Blauschiefer,93,50.0,Ahr,Anbaugebiet,Spätburgunder,Meyer-Näkel,Meyer-Näkel 2013 Blauschiefer Spätburgunder (Ahr)
214640,Germany,Intensely ripe black cherries and plums lend h...,Erzeugerabfüllung Trocken,90,31.0,Ahr,Anbaugebiet,Spätburgunder,Winzergenossenschaft Mayschoss-Altenahr,Winzergenossenschaft Mayschoss-Altenahr 2011 E...
229612,Germany,"Plump, fleshy black cherry and berry flavors c...",S,93,85.0,Ahr,Anbaugebiet,Spätburgunder,Meyer-Näkel,Meyer-Näkel 2013 S Spätburgunder (Ahr)
231902,Germany,"Ripe, almost ruddy, notes of blackberry and ch...",XII Trauben Trocken,92,62.0,Ahr,Anbaugebiet,Spätburgunder,Winzergenossenschaft Mayschoss-Altenahr,Winzergenossenschaft Mayschoss-Altenahr 2012 X...


In [206]:
df[(df['country']=='Germany') & (df['province']=='Rheingau')].fillna('Rheingau')

,country,description,designation,points,price,province,region_1,variety,winery,title
732,Germany,"Whiffs of spice, honeycomb and lanolin meld cu...",Hochheimer Domdechaney Trocken,91,36.0,Rheingau,Rheingau,Riesling,Domdechant Werner,Domdechant Werner 2012 Hochheimer Domdechaney ...
734,Germany,"Lavish aromas of ripe peach, canteloupe and ho...",Medium Dry Kabinett,91,25.0,Rheingau,Rheingau,Riesling,Schloss Vollrads,Schloss Vollrads 2012 Medium Dry Kabinett Ries...
1344,Germany,Laser-edged mineral tones cut a crystalline ed...,Kabinett,90,30.0,Rheingau,Rheingau,Riesling,Robert Weil,Rheingau
1961,Germany,"Lushly textured and expansive on the palate, t...",Hochheimer Kirchenstück Trocken,90,35.0,Rheingau,Rheingau,Riesling,Domdechant Werner,Domdechant Werner 2012 Hochheimer Kirchenstück...
2046,Germany,Steel and stones are brightened by tones of le...,Trocken,90,20.0,Rheingau,Rheingau,Riesling,Robert Weil,Robert Weil 2014 Trocken Riesling (Rheingau)
...,...,...,...,...,...,...,...,...,...,...
233015,Germany,Breathtaking scents of white peach and blossom...,Spätlese,93,31.0,Rheingau,Rheingau,Riesling,Schloss Vollrads,Schloss Vollrads 2014 Spätlese Riesling (Rhein...
233512,Germany,Whiffs of green herbs and nuts lend savoriness...,Dry,87,18.0,Rheingau,Rheingau,Riesling,Hans Lang,Hans Lang 2013 Dry Riesling (Rheingau)
234326,Germany,"Rose petals and violet perfume this fragrant, ...",Rüdesheim Berg Roseneck Spätburgunder Trocken,90,18.0,Rheingau,Rheingau,Pinot Noir,Carl Ehrhard,Carl Ehrhard 2013 Rüdesheim Berg Roseneck Spät...
234503,Germany,More delicate in character than the Kiedrich T...,Kiedrich Gräfenberg Auslese,93,94.0,Rheingau,Rheingau,Riesling,Robert Weil,Robert Weil 2014 Kiedrich Gräfenberg Auslese R...


In [207]:

df[(df['country']=='Germany') & (df['province']=='Nahe')].fillna('Anbaugebiet')

,country,description,designation,points,price,province,region_1,variety,winery,title
735,Germany,Zesty lime and evergreen notes mingle with lus...,Löhrer Berg Spätlese,91,21.0,Nahe,Anbaugebiet,Scheurebe,Bürgermeister Willi Schweinhardt,Anbaugebiet
2691,Germany,"Dry, bold and intensely mineral, this powerful...",Felseneck Wallhausen Trocken GG,90,68.0,Nahe,Anbaugebiet,Riesling,Prinz Salm,Prinz Salm 2011 Felseneck Wallhausen Trocken ...
3030,Germany,Unusual notes of lemongrass and pine lend a cu...,Monzinger Frühlingsplätzchen Auslese,88,25.0,Nahe,Anbaugebiet,Riesling,Weingut Weber,Weingut Weber 2013 Monzinger Frühlingsplätzche...
3223,Germany,"While dry in style, this late-harvest Riesling...",Niederhäuser Hermannshöhle Spätlese Dry Gutsab...,90,38.0,Nahe,Anbaugebiet,Riesling,Mathern,Anbaugebiet
3224,Germany,"While dry in style, this late-harvest Riesling...",Niederhäuser Hermannshöhle Spätlese Dry Gutsab...,90,38.0,Nahe,Anbaugebiet,Riesling,Mathern,Anbaugebiet
...,...,...,...,...,...,...,...,...,...,...
230261,Germany,"More mineral than fruity, this dry Riesling ha...",Anbaugebiet,90,Anbaugebiet,Nahe,Anbaugebiet,Riesling,Schlossgut Diel,Schlossgut Diel 2014 Riesling (Nahe)
230922,Germany,Perfumed Mandarin orange and peach flavors abo...,Kreuznacher St Martin Spätlese,89,22.0,Nahe,Anbaugebiet,Gewürztraminer,SJ Montigny,SJ Montigny 2015 Kreuznacher St Martin Spätles...
231606,Germany,A crush of wet slate persists throughout this ...,Wallhäuser Johannisberg GG Erste Lage Trocken,92,70.0,Nahe,Anbaugebiet,Riesling,Prinz Salm,Prinz Salm 2014 Wallhäuser Johannisberg GG Ers...
233023,Germany,White flowers and lime skins juxtapose smoke a...,Oberhäuser Leistenberg Kabinett,93,30.0,Nahe,Anbaugebiet,Riesling,Dönnhoff,Dönnhoff 2014 Oberhäuser Leistenberg Kabinett ...


In [208]:
df[(df['country']=='Germany') & (df['province']=='Baden')].fillna('Baden-Wurttemberg')


,country,description,designation,points,price,province,region_1,variety,winery,title
1114,Germany,"Wildflower and lemon zest brighten this dry, c...",Schwarzer Adler,89,25.0,Baden,Baden-Wurttemberg,Pinot Gris,Franz Keller,Baden-Wurttemberg
1788,Germany,Pristine white peaches seem suspended in a cry...,Schwarzer Adler,90,25.0,Baden,Baden-Wurttemberg,Pinot Blanc,Franz Keller,Baden-Wurttemberg
1789,Germany,Pristine white peaches seem suspended in a cry...,Schwarzer Adler,90,25.0,Baden,Baden-Wurttemberg,Pinot Blanc,Franz Keller,Baden-Wurttemberg
2656,Germany,"Profoundly floral, this aromatic white balance...",Golf Trocken,88,16.0,Baden,Baden-Wurttemberg,Rivaner,Johannes Deppisch,Johannes Deppisch 2012 Golf Trocken Rivaner (B...
2712,Germany,"This juicy, fruit-forward Pinot Noir has fresh...",Franz Anton Schwarzer Adler,90,42.0,Baden,Baden-Wurttemberg,Pinot Noir,Franz Keller,Franz Keller 2011 Franz Anton Schwarzer Adler ...
...,...,...,...,...,...,...,...,...,...,...
222988,Germany,"Robust in body yet squarely structured, the Fr...",Franz Anton Schwarzer Adler,94,42.0,Baden,Baden-Wurttemberg,Pinot Noir,Franz Keller,Franz Keller 2014 Franz Anton Schwarzer Adler ...
224032,Germany,Rose petal and violets persist throughout this...,Bienenberg R Erste Lage GG,95,103.0,Baden,Baden-Wurttemberg,Spätburgunder,Bernhard Huber,Bernhard Huber 2010 Bienenberg R Erste Lage GG...
224051,Germany,Another stand out among a lineup of stunning G...,Blauer Schulen,95,41.0,Baden,Baden-Wurttemberg,Spätburgunder,Ziereisen,Ziereisen 2010 Blauer Schulen Spätburgunder (B...
227208,Germany,It's difficult to name favorites amidst this v...,Schlossberg Erste Lage R GG,91,117.0,Baden,Baden-Wurttemberg,Spätburgunder,Bernhard Huber,Bernhard Huber 2011 Schlossberg Erste Lage R G...


In [209]:

df[(df['country']=='Germany') & (df['province']=='Franken')].fillna('Franken')

,country,description,designation,points,price,province,region_1,variety,winery,title
1708,Germany,A fresh green-floral touch persists throughout...,Gutswein Trocken,90,22.0,Franken,Franken,Silvaner,Rainer Sauer,Rainer Sauer 2014 Gutswein Trocken Silvaner (F...
3509,Germany,"Woody, smoky notes belie the intensely fruity ...",Casteller Kugelspiel Spätlese,90,35.0,Franken,Franken,Rieslaner,Castell,Franken
5380,Germany,Whiffs of smoke and grain lend a savory bent t...,Casteller Schlossberg Trocken GG,88,76.0,Franken,Franken,Riesling,Castell,Franken
19725,Germany,"A selection from the Würzburger Innere Leiste,...",Montonia,87,65.0,Franken,Franken,Spätburgunder,Weingut Am Stein,Weingut Am Stein 2007 Montonia Spätburgunder (...
19726,Germany,"A selection from the Würzburger Innere Leiste,...",Montonia,87,65.0,Franken,Franken,Spätburgunder,Weingut Am Stein,Weingut Am Stein 2007 Montonia Spätburgunder (...
...,...,...,...,...,...,...,...,...,...,...
215056,Germany,"Surprisingly tropical, with concentrated notes...",Escherndorfer Lump Kabinett Trocken,88,21.0,Franken,Franken,Silvaner,Michael Fröhlich,Michael Fröhlich 2012 Escherndorfer Lump Kabin...
218603,Germany,Dusty dried herbs accent bright notes of tange...,Casteller Schlossberg Trocken GG,89,76.0,Franken,Franken,Silvaner,Castell,Castell 2012 Casteller Schlossberg Trocken GG ...
225317,Germany,"This dry Silvaner balances a crisp, lean fruit...",Trocken,88,18.0,Franken,Franken,Silvaner,Castell,Castell 2011 Trocken Silvaner (Franken)
227155,Germany,"Pleasant pear and melon notes, a light- to med...",Trocken,84,20.0,Franken,Franken,Riesling,Manfried,Manfried 2009 Trocken Riesling (Franken)


In [210]:
df[(df['country']=='Germany') & (df['province']=='Pfalz')].fillna('Renanina-Palatinado')


,country,description,designation,points,price,province,region_1,variety,winery,title
1996,Germany,"Refreshing and full of vitality, this Pinot Gr...",Renanina-Palatinado,88,12.0,Pfalz,Renanina-Palatinado,Pinot Gris,Villa Wolf,Renanina-Palatinado
2423,Germany,"Stridently purple in hue, this bold, richly be...",Cuvée Noir,90,21.0,Pfalz,Renanina-Palatinado,Red Blend,Pflüger,Renanina-Palatinado
2540,Germany,"While dry in style and crisply balanced, plump...",Wachenheim Trocken,90,18.0,Pfalz,Renanina-Palatinado,Pinot Blanc,Fitz-Ritter,Fitz-Ritter 2014 Wachenheim Trocken Pinot Blan...
2541,Germany,"While dry in style and crisply balanced, plump...",Wachenheim Trocken,90,18.0,Pfalz,Renanina-Palatinado,Pinot Blanc,Fitz-Ritter,Fitz-Ritter 2014 Wachenheim Trocken Pinot Blan...
2661,Germany,Ripe melon and grapefruit flavors are plush an...,Renanina-Palatinado,88,12.0,Pfalz,Renanina-Palatinado,Riesling,Saint M,Saint M 2013 Riesling (Pfalz)
...,...,...,...,...,...,...,...,...,...,...
233414,Germany,Starts off with aromas of green gage plum and ...,Im Sonnenschein Grosses Gewächs,91,90.0,Pfalz,Renanina-Palatinado,Riesling,Ökonomierat Rebholz,Ökonomierat Rebholz 2007 Im Sonnenschein Gross...
233428,Germany,From an old clone yielding smaller berries and...,Forster Kichenstück Cyriakus Spätlese Trocken,91,41.0,Pfalz,Renanina-Palatinado,Riesling,Eugen Müller,Eugen Müller 2007 Forster Kichenstück Cyriakus...
233429,Germany,"This is a rich, full-bodied dry Riesling. Hone...",Forster Jesuitengarten Grosses Gewächs Spätles...,91,57.0,Pfalz,Renanina-Palatinado,Riesling,Geh. Rat Dr. von Bassermann-Jordan,Geh. Rat Dr. von Bassermann-Jordan 2006 Forste...
233430,Germany,"This sweet, luscious Riesling shows an incredi...",Deidesheimer Hohenmorgen Auslese,91,Renanina-Palatinado,Pfalz,Renanina-Palatinado,Riesling,Geh. Rat Dr. von Bassermann-Jordan,Geh. Rat Dr. von Bassermann-Jordan 2007 Deides...


In [211]:
df[(df['country']=='Germany') & (df['province']=='Mosel-Saar-Ruwer')].fillna('Mosel-Saar-Ruwer')


,country,description,designation,points,price,province,region_1,variety,winery,title
19746,Germany,From a vintage not known for its supersweet se...,Leiwener Laurentiuslay Beerenauslese,96,175.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,St. Urbans-Hof,St. Urbans-Hof 2008 Leiwener Laurentiuslay Bee...
19747,Germany,From a vintage not known for its supersweet se...,Leiwener Laurentiuslay Beerenauslese,96,175.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,St. Urbans-Hof,St. Urbans-Hof 2008 Leiwener Laurentiuslay Bee...
19748,Germany,From a vintage not known for its supersweet se...,Leiwener Laurentiuslay Beerenauslese,96,175.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,St. Urbans-Hof,St. Urbans-Hof 2008 Leiwener Laurentiuslay Bee...
19938,Germany,"This is a fairly full-bodied Mosel auslese, wi...",Piesporter Goldtröpfchen Auslese Goldcap,91,46.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,Johann Haart,Johann Haart 2006 Piesporter Goldtröpfchen Aus...
19939,Germany,"This is a fairly full-bodied Mosel auslese, wi...",Piesporter Goldtröpfchen Auslese Goldcap,91,46.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,Johann Haart,Johann Haart 2006 Piesporter Goldtröpfchen Aus...
...,...,...,...,...,...,...,...,...,...,...
210294,Germany,"This elegant, medium-bodied kabinett boasts te...",Ehlen Kabinett*,88,20.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,Stephan Ehlen,Stephan Ehlen 2006 Ehlen Kabinett* Riesling (M...
214970,Germany,"Atypically plump and creamy in the mouth, with...",Trittenheimer Kabinett,87,25.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,Grans-Fassian,Grans-Fassian 2007 Trittenheimer Kabinett Ries...
228657,Germany,"Unlike many of the kabinetts from 2006, this i...",Piesporter Goldtröpfchen Kabinett,86,19.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,St. Urbans-Hof,St. Urbans-Hof 2006 Piesporter Goldtröpfchen K...
228661,Germany,"Sweet and rich, verging on heavy, this is a ra...",Bernkasteler alte Badstube am Doctorberg Kabinett,86,24.0,Mosel-Saar-Ruwer,Mosel-Saar-Ruwer,Riesling,Dr. Pauly Bergweiler,Dr. Pauly Bergweiler 2006 Bernkasteler alte Ba...


In [212]:

df[(df['country']=='Germany') & (df['province']=='Rhein-Mosel')].fillna('Mosel-Saar-Ruwer')


,country,description,designation,points,price,province,region_1,variety,winery,title
90790,Germany,It's an intriguing idea to blend fruit from th...,Detonation,84,11.0,Rhein-Mosel,Mosel-Saar-Ruwer,Riesling,Immich-Batterieberg,Mosel-Saar-Ruwer
90791,Germany,It's an intriguing idea to blend fruit from th...,Detonation,84,11.0,Rhein-Mosel,Mosel-Saar-Ruwer,Riesling,Immich-Batterieberg,Mosel-Saar-Ruwer


In [213]:
df[(df['country']=='Germany') & (df['province']=='Mittelrhein')].fillna('Renanina-Palatinado')


,country,description,designation,points,price,province,region_1,variety,winery,title
166326,Germany,Rippling layers of candle wax and honey lend r...,Alte Reben Bopparder Hamm Feinherb,94,33.0,Mittelrhein,Renanina-Palatinado,Riesling,Matthias Müller,Matthias Müller 2012 Alte Reben Bopparder Hamm...
213622,Germany,Piercing lemon-lime acidity cuts a razor's edg...,Hahnenhof Devon S Spätlese Trocken,91,Renanina-Palatinado,Mittelrhein,Renanina-Palatinado,Riesling,Toni Jost,Toni Jost 2012 Hahnenhof Devon S Spätlese Troc...
214056,Germany,Opulent white blossom and tropical fruit flavo...,S Bopparder Hamm Feuerlay Trocken,91,29.0,Mittelrhein,Renanina-Palatinado,Riesling,Matthias Müller,Matthias Müller 2012 S Bopparder Hamm Feuerlay...
214067,Germany,Delicate mango and papaya sweetness accent thi...,Bacharacher Spätlese Feinherb,91,34.0,Mittelrhein,Renanina-Palatinado,Riesling,Toni Jost,Toni Jost 2011 Bacharacher Spätlese Feinherb R...
227392,Germany,"While still vibrantly peachy and floral, four ...",Rheindiabas,93,26.0,Mittelrhein,Renanina-Palatinado,Riesling,Matthias Müller,Matthias Müller 2012 Rheindiabas Riesling (Mit...


In [214]:
df[(df['country']=='Germany') & (df['province']=='Landwein Rhein')].fillna('Anbaugebiet')

,country,description,designation,points,price,province,region_1,variety,winery,title
228465,Germany,Deeply savory tones of bramble and earth are j...,Anbaugebiet,88,10.0,Landwein Rhein,Anbaugebiet,Riesling,HXM,HXM NV Riesling (Landwein Rhein)
231925,Germany,Tones of mineral and steel hold center stage i...,Stock and Stein Trocken,90,17.0,Landwein Rhein,Anbaugebiet,Riesling,Peter Jakob Kühn,Peter Jakob Kühn 2014 Stock and Stein Trocken ...


In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234668 entries, 0 to 234667
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      234602 non-null  object 
 1   description  234668 non-null  object 
 2   designation  165151 non-null  object 
 3   points       234668 non-null  int64  
 4   price        216150 non-null  float64
 5   province     234602 non-null  object 
 6   region_1     195693 non-null  object 
 7   variety      234667 non-null  object 
 8   winery       234668 non-null  object 
 9   title        151122 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 12.5+ MB
